In [3]:
import cv2
import mediapipe as mp
import pickle
import numpy as np
from tensorflow.keras.models import load_model

In [4]:
import numpy as np
from tensorflow.keras.models import load_model

class ASLPredictor:
    def __init__(self, model_path='final_asl_model.keras', labels_path='class_labels.pkl'):
        self.model = load_model(model_path)
        with open(labels_path, 'rb') as f:
            self.class_labels = pickle.load(f)
        
    def predict(self, landmarks):
        """Predict ASL sign from normalized landmarks"""
        landmarks = np.array(landmarks).reshape(1, -1).astype('float32')
        preds = self.model.predict(landmarks, verbose=0)[0]
        top_idx = np.argmax(preds)
        return {
            'prediction': self.class_labels[top_idx],
            'confidence': float(preds[top_idx]),
            'all_predictions': list(zip(self.class_labels, preds))
        }

# Initialize predictor
predictor = ASLPredictor()

In [5]:
import cv2
import mediapipe as mp

def run_live_demo():
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
    
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
            
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            landmarks = []
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Extract and normalize landmarks
                wrist = hand_landmarks.landmark[0]
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x - wrist.x, 
                                     lm.y - wrist.y, 
                                     lm.z - wrist.z])
            
            # Make prediction
            result = predictor.predict(landmarks)
            cv2.putText(frame, 
                       f"{result['prediction'].upper()} ({result['confidence']:.0%})",
                       (50, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 4)
        
        cv2.imshow('ASL Recognition', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

# Run the demo
run_live_demo()

I0000 00:00:1744901516.761686  183877 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3 Pro
W0000 00:00:1744901516.771395  184582 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744901516.778781  184583 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [5]:
model = load_model('final_asl_model.keras')


In [ ]:
print("summary of asl model")
model.summary()

summary of asl model


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_43 (Dense)                │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 36)             │         4,644 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 239,342 (934.93 KB)

 Trainable params: 79,524 (310.64 KB)

 Non-trainable params: 768 (3.00 KB)

 Optimizer params: 159,050 (621.29 KB)

: 

In [ ]:
import cv2
import mediapipe as mp

def run_live_demo():
    mp_hands = mp.solutions.hands
    mp_face = mp.solutions.face_mesh

    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
    face_mesh = mp_face.FaceMesh(static_image_mode=False)

    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
            
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        hand_results = hands.process(rgb_frame)
        face_results = face_mesh.process(rgb_frame)
        
        if hand_results.multi_hand_landmarks:
            landmarks = []
            for hand_landmarks in hand_results.multi_hand_landmarks:
                # Draw landmarks
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Extract and normalize landmarks
                wrist = hand_landmarks.landmark[0]
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x - wrist.x, 
                                     lm.y - wrist.y, 
                                     lm.z - wrist.z])
            
            # Make prediction
            result = predictor.predict(landmarks)
            cv2.putText(frame, 
                       f"{result['prediction'].upper()} ({result['confidence']:.0%})",
                       (50, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 4)

        if face_results.multi_face_landmarks:
            for face_landmarks in face_results.multi_face_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, face_landmarks, mp_face.FACEMESH_TESSELATION)
        
        cv2.imshow('ASL Recognition', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

# Run the demo
run_live_demo()


I0000 00:00:1744901636.293952  183877 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3 Pro
W0000 00:00:1744901636.301419  187743 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1744901636.302098  183877 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3 Pro
W0000 00:00:1744901636.304646  187755 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744901636.308349  187743 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744901636.309874  187755 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


: 